# <center>Image DeBlur



### Import Libraries

In [0]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, InputLayer, Conv2D,UpSampling2D , Flatten,MaxPooling2D,Conv2DTranspose
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from PIL import Image
import random
from math import ceil

### Set Filepath for Dataset

In [0]:
os.chdir('CERTH_ImageBlurDataset') #enter filepath of dataset
os.curdir   #enter dataset directory
os.listdir()    #list contents of current directory

['Artificially-Blurred', 'Naturally-Blurred', 'Undistorted']

### Dataset preperation

#### Sort Out Fake Images

In [0]:
init_size=100
folders=os.listdir()
filelist=[]
fake_data=[]
real_data=[]
for i in folders[0:3]:
    files=os.listdir(i)
    for j in files:
        im = Image.open(i+'\\'+j) # opening each image
        width = 50   #setting width of image
        height = 50 #setting height of image
        im5 = im.resize((width, height), Image.ANTIALIAS)   #resizing image
        x=np.asarray(im5)  #convert image to array
        x =(x-x.mean())/255.0# best down-sizing filter
        fake_data.append(x)

#### Sort out Real Images

In [0]:
for i in folders[2:]:
    files1=os.listdir(i)
    for j in files1:
        im = Image.open(i+'\\'+j) 
        width = 50
        height = 50
        im5 = im.resize((width, height), Image.LANCZOS)  
        x=np.asarray(im5)
        x =(x-x.mean())/255.0# best down-sizing filter
        real_data.append(x)

### Exploratory Data Analysis (EDA)

##### View Fake images:

In [0]:
fake_data= np.asarray(fake_data)     #convert image to array
plt.imshow(fake_data[70], interpolation='nearest') #setting the interpolation to 'nearest' displays an image without trying to interpolate between pixels if the display resolution is not the same as the image resolution.

##### View Real Images:

In [0]:
real_data= np.asarray(real_data)      #convert image to array
plt.imshow(real_data[50], interpolation='nearest') #setting the interpolation to 'nearest' displays an image without trying to interpolate between pixels if the display resolution is not the same as the image resolution.

### Structure the Model

##### Define Optomizer

In [0]:
def adam_optimizer():
    return Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

##### Structure Generator

In [0]:
def create_generator():
    generator=tf.keras.models.Sequential()
    generator.add(InputLayer(input_shape=(50,100,100)))
    
    generator.add(Conv2D(32, (2, 2), activation='tanh', padding='same', strides=2))
    generator.add(layers.LeakyReLU(0.6))
    generator.add(layers.Dropout(0.4))
    
    generator.add(Conv2D(32, (2, 2), activation='tanh', padding='same'))
    generator.add(layers.LeakyReLU(0.3))
    generator.add(layers.Dropout(0.2))
    
    generator.add(Conv2D(32, (3, 3), activation='tanh', padding='same'))
    generator.add(UpSampling2D((2, 1)))
    generator.add(Conv2D(3, (5, 5), activation='tanh', padding='same'))
    
    generator.add(layers.Dense(units=3, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator

g=create_generator()
g.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 25, 50, 32)        12832     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 25, 50, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 25, 50, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 25, 50, 32)        4128      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 25, 50, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 25, 50, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 25, 50, 32)        9248      
__________

##### Structure Discriminator

In [0]:
def create_discriminator():
    discriminator=tf.keras.models.Sequential()
    discriminator.add(InputLayer(input_shape=(50,50,3)))
    
    discriminator.add(Conv2D(10, (2, 2), activation='tanh', padding='same', strides=2))
    discriminator.add(layers.Dense(units=100))
    discriminator.add(layers.LeakyReLU(0.2))
    discriminator.add(layers.Dropout(0.3))
    
    discriminator.add(Conv2D(10, (3, 3), activation='tanh', padding='same', strides=2))
    discriminator.add(layers.Dense(units=50))
    discriminator.add(layers.LeakyReLU(0.2))
    discriminator.add(layers.Dropout(0.3))
    
    discriminator.add(Conv2D(10, (3, 3), activation='tanh', padding='same', strides=2))
    discriminator.add(layers.Dense(units=25))
    discriminator.add(layers.LeakyReLU(0.2))
    discriminator.add(layers.Dropout(0.3))
    
    discriminator.add(Conv2D(1, (4, 4), activation='sigmoid', padding='same', strides=3))  
    #discriminator.add(Conv2D(1, (4, 4), activation='tanh', padding='same')) 
    discriminator.add(MaxPooling2D(pool_size = (2, 3)))
    #discriminator.add(Conv2DTranspose(1, (2,2), strides=(2,2)))
    discriminator.add(Flatten())
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator

d =create_discriminator()
d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 25, 25, 10)        130       
_________________________________________________________________
dense_5 (Dense)              (None, 25, 25, 100)       1100      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 25, 25, 100)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 25, 25, 100)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 10)        9010      
_________________________________________________________________
dense_6 (Dense)              (None, 13, 13, 50)        550       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 13, 13, 50)        0         
__________

### Combine VGG16

In [0]:
def create_gan(discriminator, generator):
    d.trainable=False     #This enables us to treat the model as a combination of our custom GAN and the VGG16
    gan_input = Input(shape=(None,100,100))   #set the input shape.
    x = g(gan_input)
    gan_output= d(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

### Create GAN

In [0]:
gan = create_gan(d,g)
gan.summary()  #view the structure to ensure it is correct.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, None, 100, 100)    0         
_________________________________________________________________
sequential_2 (Sequential)    multiple                  28623     
_________________________________________________________________
sequential_3 (Sequential)    multiple                  15976     
Total params: 44,599
Trainable params: 28,623
Non-trainable params: 15,976
_________________________________________________________________
0
1


### Input Preperation

In [0]:
i=0     #set counter
epoch_num=1   #set number of epochs
batches=2     #set number of batches
for epoch in range(epoch_num):
    i=i+1
    for index in range(batches):
        # [Batch Preparation]
        print(index)
        noise= np.random.normal(0,1, [batches,50,100,100])
        noise = tf.cast(noise, tf.float32)
    
        # Generate fake inputs
        gen_images = g.predict(x=noise,steps=10)
        y_gen = np.ones(gen_images.shape[0])  
        ran_real_image =real_data[np.random.randint(low=0,high=real_data.shape[0],size=batches)] #get random set of real images
        ran_fake_image =real_data[np.random.randint(low=0,high=real_data.shape[0],size=batches)]#get random set of fake images

### Obtain Predictions:

In [0]:
 #Construct different batches of  real and fake data 
X= np.concatenate([ran_real_image, ran_fake_image])
y_combined=np.zeros(2*batches)
y_combined[:batches]=0.9
        
d.trainable=True   #Train only the custom GAN.  
d.train_on_batch(X, y_combined)
                
noise= np.random.randint(0,1, [batches,50,100,100])
noise = tf.cast(noise, tf.float32)
y_gen = np.ones(batches)
d.trainable=False
gan.train_on_batch(noise, y_gen)
noise= np.random.randint(0,1, [batches,50,100,100])
noise = tf.cast(noise, tf.float32)
gen_images = g.predict(x=noise,steps=10)
gen_images = gen_images

### View Images:

In [0]:
dim=(20,20)
figsize=(20,20)    
plt.figure(figsize=figsize)
for i in range(gen_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(gen_images[i]*256, interpolation='nearest')
    plt.axis('off')
    plt.tight_layout()    
    os.chdir('output')
    plt.savefig('actual'+str(i)+'.png') 

In [0]:
# Send Output to folder
result = Image.fromarray((gen_images[5]*256).astype(np.uint8))   #obtain array form of image.
os.chdir('output')      #specify the file path to save the image.
result.save('actual.png')      #save the image
a=(gen_images[5]*255.0).astype(np.uint8)   #Unsigned Integers of 8 bits. A uint8 data type contains all whole numbers from 0 to 255.